# 🔍 SWaT 데이터셋 LSTM-AE 기반 이상 탐지 (Anomaly Detection)

---

## 📋 개요

| 항목 | 내용 |
|------|------|
| **데이터셋** | SWaT (Secure Water Treatment) - 싱가포르 iTrust 연구소 |
| **모델** | LSTM-AE (Long Short-Term Memory Autoencoder) |
| **방법** | 비지도 이상 탐지 - 정상 데이터로 학습 후 Reconstruction Error 기반 탐지 |
| **평가지표** | Precision, Recall, F1-Score, AUC-ROC |

---

## 📁 파이프라인

```
1. 환경 설정 및 라이브러리 임포트
2. 데이터 로딩
3. 데이터 전처리 (다운샘플링, 정규화, 시퀀스 생성)
4. LSTM-AE 모델 정의
5. 모델 학습
6. 이상 탐지 및 평가
7. 결과 시각화
```

---
## 1️⃣ 환경 설정 및 라이브러리 임포트

필요한 라이브러리들을 임포트하고 GPU 사용 가능 여부를 확인합니다.

In [ ]:
# ============================================================
# 라이브러리 임포트
# ============================================================

# 데이터 처리
import numpy as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리 및 평가
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, precision_recall_curve, roc_curve,
    confusion_matrix, classification_report
)

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 유틸리티
from tqdm.notebook import tqdm  # Jupyter용 프로그레스 바
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("✅ 라이브러리 임포트 완료!")

In [ ]:
# ============================================================
# GPU/CPU 디바이스 설정
# ============================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"🖥️ 사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"   GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"   GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

---
## 2️⃣ 하이퍼파라미터 설정

전처리, 모델, 학습에 사용할 하이퍼파라미터를 정의합니다.

### 주요 파라미터 설명

| 파라미터 | 설명 | 권장값 |
|----------|------|--------|
| `downsample_rate` | 데이터 축소 비율 (10 = 1/10) | 10~20 |
| `window_size` | 시퀀스 길이 (타임스텝 수) | 50~200 |
| `stride` | 윈도우 이동 간격 | window_size의 10~20% |
| `hidden_dim` | LSTM 은닉층 차원 | 64~256 |
| `latent_dim` | 잠재 공간 차원 | 32~128 |
| `num_layers` | LSTM 레이어 수 | 1~3 |
| `threshold_percentile` | 이상 판정 임계값 백분위 | 95~99 |

In [ ]:
# ============================================================
# 하이퍼파라미터 설정
# ============================================================

CONFIG = {
    # ----- 데이터 경로 -----

    
    # ----- 전처리 파라미터 -----

    
    # ----- 시퀀스 파라미터 -----

    
    # ----- LSTM-AE 모델 파라미터 -----

    
    # ----- 학습 파라미터 -----

    
    # ----- 이상 탐지 파라미터 -----

}

# 설정 출력


---
## 3️⃣ 데이터 로딩

SWaT 데이터셋을 로딩합니다.

### 데이터셋 구성

| 파일 | 설명 | 내용 |
|------|------|------|
| `normal.csv` | 정상 운영 데이터 | 7일간 정상 운영 기록 |
| `attack.csv` | 공격 포함 테스트 데이터 | 4일간 36개 공격 시나리오 포함 |

### 컬럼 구성 (51개 센서 + 레이블)
- **FIT**: Flow meter
- **LIT**: Level transmitter  
- **AIT**: Analyzer indicator transmitter
- **MV**: Motorized valve
- **P**: Pump
- **UV**: UV dechlorinator
- **PIT**: Pressure indicator transmitter
- **DPIT**: Differential pressure indicator transmitter

In [ ]:
# ============================================================
# 데이터 로딩
# ============================================================


In [ ]:
# ============================================================
# 데이터 미리보기
# ============================================================


In [ ]:
# ============================================================
# 컬럼명 정리 및 레이블 추출 (SWaT 데이터셋용)
# ============================================================

# 컬럼명 정리 (공백 제거)


# 레이블 컬럼명 확인


# 공격 레이블 추출 ('Normal' → 0, 'Attack' → 1)


# 레이블 컬럼 제거


---
## 4️⃣ 데이터 전처리

### 전처리 단계

1. **수치형 컬럼 선택**: Timestamp 등 비수치형 제거
2. **다운샘플링**: 데이터 크기 축소 (1초 → 10초 간격)
3. **결측치 처리**: 전방/후방 채움 + 평균값 대체
4. **저분산 컬럼 제거**: 분산이 거의 0인 컬럼 제거
5. **고상관 컬럼 제거**: 상관계수 0.95 이상인 중복 컬럼 제거
6. **정규화**: MinMax Scaling (0-1)

In [ ]:
# ============================================================
# Step 1: 수치형 컬럼만 선택
# ============================================================

# 제외할 컬럼 (비수치형) - SWaT 데이터셋용


# 수치형 데이터 추출


# 강제 수치형 변환 (문자열 → 숫자, 변환 불가 시 NaN)


In [ ]:
# ============================================================
# Step 2: 다운샘플링
# ============================================================

downsample_rate = CONFIG['downsample_rate']

# 매 downsample_rate 번째 행만 선택


# 레이블도 동일하게 다운샘플링


In [ ]:
# ============================================================
# Step 3: 결측치 처리
# ============================================================

# 결측치 현황 확인


# 전방/후방 채움


# 남은 NaN은 컬럼 평균으로 대체


# 전체가 NaN인 컬럼 제거


In [ ]:
# ============================================================
# Step 4: 저분산 컬럼 제거
# ============================================================

# 각 컬럼의 분산 계산


# 분산이 임계값 이상인 컬럼만 선택


# 제거된 컬럼 확인


# 적용


In [ ]:
# ============================================================
# Step 5: 고상관 컬럼 제거 (선택적)
# ============================================================


In [ ]:
# ============================================================
# Step 6: 정규화 (MinMax Scaling)
# ============================================================

# 정상 데이터로 스케일러 학습


# 공격 데이터에 동일한 스케일러 적용


# 피처 컬럼명 저장


---
## 5️⃣ 시퀀스 생성 (Sliding Window)

시계열 데이터를 LSTM-AE 모델에 입력하기 위해 슬라이딩 윈도우 방식으로 시퀀스를 생성합니다.

### 시퀀스 생성 방식

```
원본: [x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, ...]

window_size=4, stride=2 인 경우:
  시퀀스 1: [x1, x2, x3, x4]
  시퀀스 2: [x3, x4, x5, x6]
  시퀀스 3: [x5, x6, x7, x8]
  ...
```

In [ ]:
# ============================================================
# 시퀀스 생성 함수
# ============================================================

def create_sequences(data, window_size, stride):
    """
    슬라이딩 윈도우로 시퀀스 생성
    
    Args:
        data: 입력 데이터 (samples, features)
        window_size: 윈도우 크기 (시퀀스 길이)
        stride: 윈도우 이동 간격
        
    Returns:
        sequences: (num_sequences, window_size, features)
    """


print("✅ 시퀀스 생성 함수 정의 완료")

In [ ]:
# ============================================================
# 학습/검증/테스트 데이터 분할 및 시퀀스 생성
# ============================================================


# ------------------------------------------------------------
# 정상 데이터에서 미리 테스트용 구간을 떼어내기
#  - normal_test: attack 데이터와 동일한 길이만큼 사용 (클래스 비율 1:1 가정)
#  - 나머지(normal_train_val)를 다시 train/val로 분할
# ------------------------------------------------------------

# ------------------------------------------------------------
# 시퀀스 생성
# ------------------------------------------------------------


# 테스트 데이터는 [정상 test + 공격]을 시간 순서대로 이어붙여서 구성


# 시퀀스 생성


In [ ]:
# ============================================================
# 테스트 레이블 생성 (윈도우 단위)
# ============================================================

# ------------------------------------------------------------
# 원시 시계열 단위 라벨 생성
#  - normal_test 구간: 모두 정상(0)
#  - attack 구간: 모두 공격(1)로 가정
#    (SWaT attack.csv에는 비정상 구간만 포함되어 있다고 가정)
# ------------------------------------------------------------


# ------------------------------------------------------------
# 윈도우 단위 라벨 생성
#  - 하나의 윈도우(window_size 길이)의 평균 라벨이 0.5 이상이면 공격(1)
# ------------------------------------------------------------



In [ ]:
# ============================================================
# PyTorch DataLoader 생성
# ============================================================

# Tensor 변환


# DataLoader 생성


---
## 6️⃣ LSTM-AE 모델 정의

### LSTM (Long Short-Term Memory) 특징

1. **장기 의존성 학습**: 게이트 메커니즘으로 장기 시퀀스 패턴 학습
2. **Forget Gate**: 불필요한 정보 삭제
3. **Input Gate**: 새로운 정보 추가
4. **Output Gate**: 출력할 정보 결정

### 모델 구조

```
Input (batch, seq_len, features)
    ↓
Encoder LSTM: features → hidden_dim → latent_dim
    ↓
Latent Vector (batch, latent_dim)
    ↓
Repeat Vector (batch, seq_len, latent_dim)
    ↓
Decoder LSTM: latent_dim → hidden_dim → features
    ↓
Output (batch, seq_len, features) - Reconstructed
```

In [ ]:
# ============================================================
# LSTM Encoder
# ============================================================

class LSTMEncoder(nn.Module):
    """
    LSTM Encoder
    
    - 시퀀스를 잠재 공간으로 압축
    - 마지막 타임스텝의 hidden state를 잠재 벡터로 사용
    
    Args:
        input_dim: 입력 특성 수
        hidden_dim: LSTM 은닉층 차원
        latent_dim: 잠재 공간 차원
        num_layers: LSTM 레이어 수
        dropout: 드롭아웃 비율
        bidirectional: 양방향 LSTM 사용 여부
    """
    
    def __init__():

        

        
    def forward():
        """
        Args:
            x: (batch, seq_len, input_dim)
        Returns:
            latent: (batch, latent_dim)
        """


print("✅ LSTMEncoder 정의 완료")

In [ ]:
# ============================================================
# LSTM Decoder
# ============================================================

class LSTMDecoder(nn.Module):
    """
    LSTM Decoder
    
    - 잠재 벡터로부터 원본 시퀀스 복원
    - RepeatVector로 잠재 벡터를 시퀀스 길이만큼 반복
    
    Args:
        latent_dim: 잠재 공간 차원
        hidden_dim: LSTM 은닉층 차원
        output_dim: 출력 특성 수 (= 입력 특성 수)
        seq_len: 시퀀스 길이
        num_layers: LSTM 레이어 수
        dropout: 드롭아웃 비율
    """
    
    def __init__():

        
        # Output layer
        
    def forward(self, latent):
        """
        Args:
            latent: (batch, latent_dim)
        Returns:
            output: (batch, seq_len, output_dim)
        """


print("✅ LSTMDecoder 정의 완료")

In [ ]:
# ============================================================
# LSTM Autoencoder (LSTM-AE)
# ============================================================

class LSTMAE(nn.Module):
    """
    LSTM Autoencoder
    
    이상 탐지 원리:
    - 정상 데이터로만 학습 → 정상 패턴 재구성 능력 학습
    - 이상 데이터는 재구성 오류(reconstruction error)가 높음
    
    Args:
        input_dim: 입력 특성 수
        hidden_dim: LSTM 은닉층 차원
        latent_dim: 잠재 공간 차원
        seq_len: 시퀀스 길이
        num_layers: LSTM 레이어 수
        dropout: 드롭아웃 비율
        bidirectional: 인코더 양방향 LSTM 사용 여부
    """
    
    def __init__():

        
    def forward(self, x):
        """
        Args:
            x: 입력 시퀀스 (batch, seq_len, features)
        Returns:
            reconstructed: 재구성된 시퀀스 (batch, seq_len, features)
        """

    
    def get_reconstruction_error(self, x):
        """재구성 오류 계산"""

    
    def get_latent(self, x):
        """잠재 벡터 추출"""


print("✅ LSTMAE 정의 완료")

In [ ]:
# ============================================================
# 모델 인스턴스 생성
# ============================================================

model = 

# 모델 구조 출력


# 파라미터 수 계산


---
## 7️⃣ 모델 학습

### 학습 전략

- **손실 함수**: MSE (Mean Squared Error) - 재구성 손실
- **옵티마이저**: Adam with weight decay (L2 정규화)
- **Early Stopping**: 검증 손실이 개선되지 않으면 조기 종료
- **Gradient Clipping**: 그래디언트 폭발 방지 (LSTM에서 특히 중요)

In [ ]:
# ============================================================
# 학습 설정
# ============================================================



# 학습 기록


In [ ]:
# ============================================================
# 학습 함수 정의
# ============================================================

def train_epoch():
    """한 에폭 학습"""



def validate():
    """검증 손실 계산"""


print("✅ 학습/검증 함수 정의 완료")

In [ ]:
# ============================================================
# 모델 학습 실행
# ============================================================

print("🚀 모델 학습 시작!")
print("=" * 60)

epochs = CONFIG['epochs']
patience = CONFIG['patience']

best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

# 학습 루프
for epoch in tqdm(range(epochs), desc="Training"):
    # 학습

    
    # 검증

    
    # Early Stopping 체크

    
    # 진행 상황 출력 (매 10 에폭)

    
    # Early Stopping


# Best 모델 복원


print(f"\n✅ 학습 완료!")
print(f"   - Best Validation Loss: {best_val_loss:.6f}")

In [ ]:
# ============================================================
# 학습 곡선 시각화
# ============================================================


---
## 8️⃣ 이상 탐지 및 평가

### 이상 탐지 원리

1. 정상 데이터로 학습된 모델은 **정상 패턴**을 잘 재구성함
2. **이상 데이터**는 학습하지 않은 패턴이므로 재구성 오류가 높음
3. 재구성 오류가 **임계값(threshold)** 이상이면 이상으로 판정

### 임계값 결정

검증 데이터(정상)의 재구성 오류 분포에서 상위 1% (99 percentile)를 임계값으로 설정

In [ ]:
# ============================================================
# 이상 점수 (Reconstruction Error) 계산 함수
# ============================================================

def compute_anomaly_scores(model, dataloader, device):
    """
    이상 점수 계산
    
    Args:
        model: 학습된 LSTM-AE 모델
        dataloader: 데이터 로더
        device: 디바이스
        
    Returns:
        scores: 각 샘플의 재구성 오류 (이상 점수)
    """


print("✅ 이상 점수 계산 함수 정의 완료")

In [ ]:
# ============================================================
# 이상 점수 계산
# ============================================================

print("⏳ 이상 점수 계산 중...")

# 검증 데이터 (정상)의 이상 점수


# 테스트 데이터의 이상 점수


In [ ]:
# ============================================================
# 임계값 결정
# ============================================================

# 검증 데이터(정상)의 상위 percentile을 임계값으로 설정


# 다양한 percentile 값 확인


In [ ]:
# ============================================================
# 예측 및 성능 평가
# ============================================================

# 예측 (점수가 임계값 이상이면 이상으로 판정)


# 성능 지표 계산


# Confusion Matrix


In [ ]:
# ============================================================
# 상세 분류 보고서
# ============================================================



---
## 9️⃣ 결과 시각화

### 시각화 항목

1. **이상 점수 분포**: 정상 vs 공격 데이터의 점수 분포
2. **ROC Curve**: True Positive Rate vs False Positive Rate
3. **Precision-Recall Curve**: 정밀도-재현율 곡선
4. **시간별 이상 점수**: 테스트 기간 동안의 이상 점수 변화

In [ ]:
# ============================================================
# 결과 시각화
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# ----- (1) 이상 점수 분포 -----


# ----- (2) ROC Curve -----


# ----- (3) Precision-Recall Curve -----


# ----- (4) 시간에 따른 이상 점수 -----


# 실제 공격 구간 표시


In [ ]:
# ============================================================
# Confusion Matrix 히트맵
# ============================================================


---
## 🔟 모델 저장

학습된 모델과 설정을 저장합니다.

In [ ]:
# ============================================================
# 모델 저장
# ============================================================

save_dict = {

}


---
## 📝 최종 결과 요약

In [ ]:
# ============================================================
# 최종 결과 요약
# ============================================================

print("=" * 60)
print("📊 SWaT LSTM-AE 이상 탐지 - 최종 결과 요약")
print("=" * 60)

print(f"\n🔧 데이터 설정:")
print(f"   - 다운샘플링: 1/{CONFIG['downsample_rate']}")
print(f"   - 피처 수: {n_features}")
print(f"   - 윈도우 크기: {CONFIG['window_size']}")
print(f"   - Stride: {CONFIG['stride']}")

print(f"\n🏗️ 모델 설정:")
print(f"   - Hidden Dim: {CONFIG['hidden_dim']}")
print(f"   - Latent Dim: {CONFIG['latent_dim']}")
print(f"   - Num Layers: {CONFIG['num_layers']}")
print(f"   - Dropout: {CONFIG['dropout']}")
print(f"   - Bidirectional: {CONFIG['bidirectional']}")

print(f"\n📈 학습 결과:")
print(f"   - 최종 학습 손실: {train_losses[-1]:.6f}")
print(f"   - 최종 검증 손실: {val_losses[-1]:.6f}")
print(f"   - Best 검증 손실: {best_val_loss:.6f}")

print(f"\n🎯 이상 탐지 성능:")
print(f"   - Threshold: {threshold:.6f}")
print(f"   - Precision: {precision:.4f}")
print(f"   - Recall: {recall:.4f}")
print(f"   - F1-Score: {f1:.4f}")
print(f"   - AUC-ROC: {auc_roc:.4f}")

print("\n" + "=" * 60)
print("✅ 완료!")
print("=" * 60)

---
## 🔄 성능 개선 방안

성능이 만족스럽지 않은 경우 다음을 시도해 보세요:

### 하이퍼파라미터 튜닝
1. `window_size` 조정: 50, 100, 200 등
2. `threshold_percentile` 조정: 95, 97, 99 등
3. `hidden_dim` 조정: 64, 128, 256 등
4. `latent_dim` 조정: 32, 64, 128 등
5. `num_layers` 조정: 1, 2, 3 등
6. `bidirectional` 활성화: 양방향 LSTM 사용

### 데이터 전처리
1. 다운샘플링 비율 조정
2. 특정 센서만 선택
3. 이동 평균 적용

### 모델 아키텍처 변형
1. GRU-AE 시도 (더 빠른 학습)
2. Attention LSTM-AE 시도
3. Variational LSTM-AE (LSTM-VAE) 시도

---
## 📚 TCN-AE vs LSTM-AE 비교

| 특성 | TCN-AE | LSTM-AE |
|------|--------|----------|
| **병렬화** | 높음 (CNN 기반) | 낮음 (순차 처리) |
| **학습 속도** | 빠름 | 느림 |
| **장기 의존성** | Dilation으로 확장 | 게이트 메커니즘 |
| **메모리 사용** | 적음 | 많음 |
| **그래디언트 흐름** | 안정적 | 소실/폭발 가능 |
| **해석 가능성** | 수용 영역 명확 | Hidden State 분석 가능 |